<a href="https://colab.research.google.com/github/DhruvaBansal00/ConfidentMT/blob/master/driver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!ls
%cd drive/My Drive/ConfidentMachineTranslation/flores
!ls
!pwd
import sys
sys.path.append('/content/drive/My Drive/ConfidentMachineTranslation/flores/src/')

drive  sample_data
/content/drive/My Drive/ConfidentMachineTranslation/flores
analysis			      FeatureSubsetBinaryClassifers.ipynb
Analysis			      FLORES.ipynb
backward_models			      LanguageAnalysis.ipynb
BoostedBinaryClassifers.ipynb	      language_models
checkpoints			      LM_Thresholding.ipynb
ClassificationDataset		      NCD_Analysis.ipynb
CompareBaselineWithClassifiers.ipynb  NNClassification.ipynb
configs				      noisychannel
data				      NoisyChannel.ipynb
data-bin			      Resources
driver.ipynb			      scripts
Ensembles			      src
Ensembling
/content/drive/My Drive/ConfidentMachineTranslation/flores


In [3]:
pip install fairseq sacrebleu sentencepiece fastBPE sacremoses

In [0]:
##Methods for CLI with fairseq - WIP

class FairseqWrapper:

    def runFairseqGenerate(self, dataFolder, sourceLang, targetLang, fwModel, beam, lenpen, dataSet, removeBpe, saveFile):
        !fairseq-generate $dataFolder --source-lang $sourceLang --target-lang $targetLang \
        --path $fwModel --beam $beam --lenpen $lenpen --gen-subset $dataSet --remove-bpe=$removeBpe --sacrebleu > $saveFile

    def runFairseqScore(self, translations, groundTruth, saveFile, metric):
        !fairseq-score --sys $translations --ref $groundTruth --$metric > $saveFile
    
    def runFairseqPreprocess(self, srcdict, preprocCommand, bpeTranslations, dest):
        !fairseq-preprocess --only-source --srcdict $srcdict --$preprocCommand $bpeTranslations \
        --destdir $dest --workers 20

    def runFairseqEvalLM(self, preprocFile, lmModel, batchSize, maxTokens, dataSet, saveFile):
        !fairseq-eval-lm $preprocFile --path $lmModel --batch-size $batchSize --max-tokens $maxTokens \
        --output-word-probs --sample-break-mode eos --gen-subset $dataSet > $saveFile

In [0]:
dataFolder = "data-bin/wiki_ne_en_bpe5000/"
fwModel = "Ensembles/April_04.pt"
bwModel = "backward_models/April_17.pt"
lmModel = "language_models/lm/checkpoint_best.pt"
sourceLang = "ne"
targetLang = "en"


In [7]:
import translationUtils
from translation import Translation

trainTranslations = translationUtils.getTranslationFromDataset("valid", fwModel, bwModel, lmModel, "ne", "en", FairseqWrapper(), dataFolder, False, True)
testTranslations = translationUtils.getTranslationFromDataset("test", fwModel, bwModel, lmModel, "ne", "en", FairseqWrapper(), dataFolder, False, True)

  0% 0/12 [00:00<?, ?it/s]/pytorch/aten/src/ATen/native/BinaryOps.cpp:66: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.
  0% 0/13 [00:00<?, ?it/s]/pytorch/aten/src/ATen/native/BinaryOps.cpp:66: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.
Namespace(align_suffix=None, alignfile=None, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='analysis/preprocessed_translations', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=1000, lr_scheduler='fixed', memory_efficient_fp16=False, min_loss_scale=0.0001, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_so

In [7]:
import dataUtils
import classification

trainTranslations = dataUtils.createObjectsFromFile("valid")
testTranslations = dataUtils.createObjectsFromFile("test")
classification.verboseTraining(trainTranslations, testTranslations, "LogisticRegression", FairseqWrapper(), 15, [0,11,12])
curvePoints = classification.precisionCurveFromClassification(trainTranslations, testTranslations, "LogisticRegression", FairseqWrapper(), [15, 16], [0,11,12])
print(curvePoints)

#################################################
Traning Logistic Regression Classifier
Train Accuracy
Correctly accepted = 0.20078740157480315
Incorrectly rejected = 0.7992125984251969
Correctly rejected = 0.9829351535836177
Incorrectly accepted = 0.017064846416382284
Total Accuracy = 0.8276670574443142
Percent acepted = 5.353653771004298
Corpus BLEU score of accepted translations = 21.61
Test Accuracy
Correctly accepted = 0.1808066759388039
Incorrectly rejected = 0.8191933240611962
Correctly rejected = 0.972117202268431
Incorrectly accepted = 0.027882797731568987
Total Accuracy = 0.7714285714285715
Percent acepted = 6.666666666666667
Corpus BLEU score of accepted translations = 21.45
#################################################



([0.0490299823633157, 0.06666666666666667], [22.44, 21.45])


In [8]:
import thresholding
curvePoints = thresholding.precisionCurveFromThresholding(testTranslations, [-0.72], 0, FairseqWrapper())
print(curvePoints)


([0.6977072310405644], [9.25])
